# W&B → Cloud Deploy Lab — Step-by-step Notebook

**Purpose:** Teach students how to train a simple model, log it to Weights & Biases (W&B) as an artifact, create a minimal inference service (FastAPI) which downloads the model artifact at startup, containerize the service, and deploy it to Google Cloud Run. The notebook is written so the training + artifact steps can be executed in Colab or a Jupyter server. Docker & cloud deploy steps are runnable on local Ubuntu lab machines.

**What you will produce:**

- A logged W&B model artifact (`iris-rf`) in project `classroom-deploy`.
- A FastAPI inference app that downloads the artifact and serves `/predict`.
- A Dockerfile to containerize the app.
- Cloud Run deployment instructions (or substitute AWS/Azure as desired).

---


## Prerequisites (before starting)

- **W&B account & API key**: Make sure students have W&B accounts (academic) and their `WANDB_API_KEY`.
- **Python**: Colab or Python 3.8+ environment.
- **For Docker & deploy**: Local Ubuntu machines with Docker and Google Cloud SDK installed.
- **Optional**: GitHub account (for CI/CD exercise).

**Notes:**
- Colab: You can run the training and artifact steps in Colab, but building Docker images and deploying with `gcloud` must be done on local machines.



## 1) Install Python dependencies

Run this cell in Colab or your local notebook to install the required packages for the lab (training, logging, and local testing).

In [42]:
!pip install --quiet wandb scikit-learn joblib numpy
print('Installed core packages (wandb, scikit-learn, joblib, numpy)')

Installed core packages (wandb, scikit-learn, joblib, numpy)


### 1.1 Set your W&B API key

There are two ways:

- **Option A — Colab / notebook only (ephemeral)**:

```python
import os
os.environ['WANDB_API_KEY'] = 'your_wandb_api_key_here'
```

- **Option B — Use `wandb login` interactively (recommended for Colab):**

```bash
!wandb login
```

- **Option C — Local Ubuntu**:

```bash
export WANDB_API_KEY='your_wandb_api_key_here'
```

Make sure `WANDB_API_KEY` is set before running W&B code.



In [43]:
import os
# Replace 'your_api_key_here' with your actual W&B API key
# You can find it at: https://wandb.ai/authorize
os.environ['WANDB_API_KEY'] = 'fbd5089fe8ff8a2f6dfc053ad7ac625ab10a7a2f'

In [44]:
# Quick check for WANDB_API_KEY
import os
if 'WANDB_API_KEY' in os.environ:
    print('WANDB_API_KEY is set (will be used by wandb.login()).')
else:
    print('WANDB_API_KEY not set. Run `os.environ[...] = '"your_key"'` in Colab or `!wandb login`.')

WANDB_API_KEY is set (will be used by wandb.login()).


## 2) Login to W&B from the notebook

This will attempt to use `WANDB_API_KEY` from the environment. If not set, `wandb.login()` may prompt you interactively (Colab).

In [45]:
import wandb
try:
    wandb.login()
    print('wandb.login() successful (or WANDB_API_KEY used).')
except Exception as e:
    print('wandb.login() failed — set WANDB_API_KEY or run `!wandb login` interactively.\n', e)

wandb.login() successful (or WANDB_API_KEY used).


In [46]:
# !wandb login --relogin

## 3) Step 1 — Train a model and log it to W&B as an artifact

This cell trains a small RandomForest on the Iris dataset, saves `model.pkl`, and logs it to W&B in project `classroom-deploy` with artifact name `iris-rf`.

**Run this cell**.

In [47]:
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
import joblib
import wandb

print('Training model...')

run = wandb.init(project='classroom-deploy', job_type='train')
print(f'\n🚀 View your run at: {run.url}')

X, y = load_iris(return_X_y=True)
clf = RandomForestClassifier(n_estimators=50, random_state=42)
clf.fit(X, y)
acc = clf.score(X, y)
print(f'Training accuracy: {acc:.4f}')

model_path = 'model.pkl'
joblib.dump(clf, model_path)

artifact = wandb.Artifact(name='iris-rf', type='model', metadata={'accuracy': acc})
artifact.add_file(model_path)
run.log_artifact(artifact)
print('Logged artifact: iris-rf')
print(f'📦 View your artifact at: {run.url}/artifacts')
run.finish()
print('\n✅ Training complete! Check the links above.')

Training model...



🚀 View your run at: https://wandb.ai/ir2023/classroom-deploy/runs/wcege67d
Training accuracy: 1.0000
Logged artifact: iris-rf
📦 View your artifact at: https://wandb.ai/ir2023/classroom-deploy/runs/wcege67d/artifacts
Logged artifact: iris-rf
📦 View your artifact at: https://wandb.ai/ir2023/classroom-deploy/runs/wcege67d/artifacts



✅ Training complete! Check the links above.


### 3.1 Verify your artifact

- Open the W&B web UI (https://wandb.ai) → your account → project `classroom-deploy` and you should see the run and the artifact.

- Optional: try to list artifacts via the API. If you encounter permission/enterprise differences, rely on the web UI.


In [48]:
import wandb
from IPython.display import display, Markdown

api = wandb.Api()
user = api.viewer
username = user.username if hasattr(user, 'username') else user.entity

links = f"""
## 🔗 Your W&B Links (Click to Open)

### Main Dashboard
- **Your W&B Home**: [https://wandb.ai/home](https://wandb.ai/home)
- **Project Dashboard**: [https://wandb.ai/{username}/classroom-deploy](https://wandb.ai/{username}/classroom-deploy)

### Artifacts
- **Artifacts Page**: [https://wandb.ai/{username}/classroom-deploy/artifacts](https://wandb.ai/{username}/classroom-deploy/artifacts)
- **Iris-RF Model**: [https://wandb.ai/{username}/classroom-deploy/artifacts/model/iris-rf](https://wandb.ai/{username}/classroom-deploy/artifacts/model/iris-rf)

### Settings
- **API Keys**: [https://wandb.ai/authorize](https://wandb.ai/authorize)
- **Account Settings**: [https://wandb.ai/settings](https://wandb.ai/settings)

**Your Username**: `{username}`  
**Artifact Reference**: `{username}/classroom-deploy/iris-rf:latest`
"""

display(Markdown(links))
print(f"\n✅ All links generated for user: {username}")


## 🔗 Your W&B Links (Click to Open)

### Main Dashboard
- **Your W&B Home**: [https://wandb.ai/home](https://wandb.ai/home)
- **Project Dashboard**: [https://wandb.ai/112201022/classroom-deploy](https://wandb.ai/112201022/classroom-deploy)

### Artifacts
- **Artifacts Page**: [https://wandb.ai/112201022/classroom-deploy/artifacts](https://wandb.ai/112201022/classroom-deploy/artifacts)
- **Iris-RF Model**: [https://wandb.ai/112201022/classroom-deploy/artifacts/model/iris-rf](https://wandb.ai/112201022/classroom-deploy/artifacts/model/iris-rf)

### Settings
- **API Keys**: [https://wandb.ai/authorize](https://wandb.ai/authorize)
- **Account Settings**: [https://wandb.ai/settings](https://wandb.ai/settings)

**Your Username**: `112201022`  
**Artifact Reference**: `112201022/classroom-deploy/iris-rf:latest`



✅ All links generated for user: 112201022


### 3.2 Generate Your W&B Links

Run this cell to see all your important W&B links that you can click to view in the browser.

In [53]:
import wandb
api = wandb.Api()
try:
    # Get your wandb username
    user = api.viewer
    username = user.username if hasattr(user, 'username') else user.entity
    print(f'Your W&B username: {username}')
    print(f'\nArtifact should be at: {username}/classroom-deploy/iris-rf')
    print(f'\nView your artifacts at: https://wandb.ai/{username}/classroom-deploy/artifacts')
    
    # Try to list artifacts
    print('\nAttempting to list artifacts...')
    artifact_full_name = f"ir2023/classroom-deploy/iris-rf"
    project_artifacts = api.artifacts(type_name='model', per_page=10,name=artifact_full_name)
    found = False
    for a in project_artifacts:
        if 'iris-rf' in a.name:
            print(f'- {a.qualified_name}')
            found = True
    if not found:
        print('No iris-rf artifacts found. Check the web UI link above.')
except Exception as e:
    print('Could not list artifacts via API. Please check the W&B web UI to inspect artifacts.')
    print(f'Visit: https://wandb.ai/home to see your projects\n', e)

Your W&B username: 112201022

Artifact should be at: 112201022/classroom-deploy/iris-rf

View your artifacts at: https://wandb.ai/112201022/classroom-deploy/artifacts

Attempting to list artifacts...
- ir2023/classroom-deploy/iris-rf:v8
- ir2023/classroom-deploy/iris-rf:v7
- ir2023/classroom-deploy/iris-rf:v6
- ir2023/classroom-deploy/iris-rf:v5
- ir2023/classroom-deploy/iris-rf:v4
- ir2023/classroom-deploy/iris-rf:v3
- ir2023/classroom-deploy/iris-rf:v2
- ir2023/classroom-deploy/iris-rf:v1
- ir2023/classroom-deploy/iris-rf:v0
- ir2023/classroom-deploy/iris-rf:v8
- ir2023/classroom-deploy/iris-rf:v7
- ir2023/classroom-deploy/iris-rf:v6
- ir2023/classroom-deploy/iris-rf:v5
- ir2023/classroom-deploy/iris-rf:v4
- ir2023/classroom-deploy/iris-rf:v3
- ir2023/classroom-deploy/iris-rf:v2
- ir2023/classroom-deploy/iris-rf:v1
- ir2023/classroom-deploy/iris-rf:v0


## 4) Step 2 — Create a FastAPI app that downloads the model artifact at startup

Run the following cell to create `app/main.py`, `app/requirements.txt`, and a `Dockerfile`. The app expects an environment variable `WANDER_MODEL_ARTIFACT` with the artifact reference, e.g. `your-username/classroom-deploy/iris-rf:latest`. If you want to bake the model into the image, see the notes below.


In [50]:
import os
os.makedirs('app', exist_ok=True)

main_py = '''import os
import joblib
import wandb
from fastapi import FastAPI
import numpy as np

app = FastAPI()
MODEL_ARTIFACT = os.environ.get('WANDB_MODEL_ARTIFACT', 'your-username/classroom-deploy/iris-rf:latest')

# Downloads the artifact and returns a loaded model
def load_model_from_wandb(artifact_ref):
    try:
        wandb.login()
    except Exception:
        pass
    api = wandb.Api()
    artifact = api.artifact(artifact_ref)
    path = artifact.download()
    model_file = os.path.join(path, 'model.pkl')
    return joblib.load(model_file)

@app.on_event('startup')
def startup():
    global model
    model = load_model_from_wandb(MODEL_ARTIFACT)

@app.get('/')
def root():
    return {'status': 'ok', 'model_artifact': MODEL_ARTIFACT}

@app.post('/predict')
def predict(features: list):
    arr = np.array(features).reshape(1, -1)
    pred = model.predict(arr)
    return {'prediction': int(pred[0])}
'''

with open('app/main.py', 'w') as f:
    f.write(main_py)

reqs = '''fastapi
uvicorn[standard]
scikit-learn
joblib
wandb
numpy
'''
with open('app/requirements.txt', 'w') as f:
    f.write(reqs)

dockerfile = '''FROM python:3.10-slim
WORKDIR /app
COPY ./requirements.txt /app/requirements.txt
RUN pip install --no-cache-dir -r /app/requirements.txt
COPY ./main.py /app/main.py
ENV PORT=8080
CMD ["uvicorn", "main:app", "--host", "0.0.0.0", "--port", "8080"]
'''
with open('app/Dockerfile', 'w') as f:
    f.write(dockerfile)

print('Wrote files to ./app:')
print(' - app/main.py')
print(' - app/requirements.txt')
print(' - app/Dockerfile')


Wrote files to ./app:
 - app/main.py
 - app/requirements.txt
 - app/Dockerfile


In [51]:
print('\n--- app/main.py ---\n')
print(open('app/main.py').read()[:1000])
print('\n(Truncated output)')


--- app/main.py ---

import os
import joblib
import wandb
from fastapi import FastAPI, Body
from typing import List
import numpy as np

app = FastAPI()
MODEL_ARTIFACT = os.environ.get('WANDB_MODEL_ARTIFACT', 'your-username/classroom-deploy/iris-rf:latest')

# Downloads the artifact and returns a loaded model
def load_model_from_wandb(artifact_ref):
    try:
        wandb.login()
    except Exception:
        pass
    api = wandb.Api()
    artifact = api.artifact(artifact_ref)
    path = artifact.download()
    model_file = os.path.join(path, 'model.pkl')
    return joblib.load(model_file)

@app.on_event('startup')
def startup():
    global model
    model = load_model_from_wandb(MODEL_ARTIFACT)

@app.get('/')
def root():
    return {'status': 'ok', 'model_artifact': MODEL_ARTIFACT}

@app.post('/predict')
def predict(features: List[float] = Body(...)):
    arr = np.array(features).reshape(1, -1)
    pred = model.predict(arr)
    return {'prediction': int(pred[0])}


(Truncated output)


### 4.1 Quick test: Download the artifact and run inference locally (no FastAPI server needed)

This lets students verify the model artifact and inference logic inside the notebook. It downloads the artifact using the W&B API and loads `model.pkl` then runs a test prediction.


In [54]:
import joblib
import wandb
import os
import numpy as np

# Get your username and construct artifact reference
api = wandb.Api()
user = api.viewer
# Use entity 'ir2023' where the artifacts are stored
entity = 'ir2023'  # Adjust this if your artifacts are under a different entity
artifact_ref = f'{entity}/classroom-deploy/iris-rf:latest'  # or use specific version like :v0

print('Attempting to download artifact:', artifact_ref)
try:
    artifact = api.artifact(artifact_ref)
    path = artifact.download()
    print('Downloaded to', path)
    print(f'\n🔗 View artifact at: https://wandb.ai/{artifact_ref.replace(":", "/files/")}')
    
    model = joblib.load(os.path.join(path, 'model.pkl'))
    sample = np.array([5.1, 3.5, 1.4, 0.2]).reshape(1, -1)
    pred = model.predict(sample)
    print(f'\n✅ Sample prediction: {pred.tolist()[0]} (Iris class)')
except Exception as e:
    print('Could not download/run model locally. Check WANDB_API_KEY and artifact ref.\n', e)
    print(f'Expected artifact format: {entity}/classroom-deploy/iris-rf:latest')

Attempting to download artifact: ir2023/classroom-deploy/iris-rf:latest


wandb:   1 of 1 files downloaded.  


Downloaded to /home/chakri/Documents/S-7/MLoPs/Mlops2025w-112201022/CLASS/Week-8/artifacts/iris-rf:v8

🔗 View artifact at: https://wandb.ai/ir2023/classroom-deploy/iris-rf/files/latest

✅ Sample prediction: 0 (Iris class)


### 4.2 Run the FastAPI app locally (non-Docker) — good for quick testing on a development machine

From the project root (where `app/` lives), run in a terminal:

```bash
# make sure WANDB_API_KEY and WANDB_MODEL_ARTIFACT are set in the environment, example:
export WANDB_API_KEY='your_api_key_here'
export WANDB_MODEL_ARTIFACT='your-username/classroom-deploy/iris-rf:latest'

# install requirements (if running outside notebook):
pip install -r app/requirements.txt

# run server
uvicorn app.main:app --host 0.0.0.0 --port 8080
```

Then in another terminal test:

```bash
curl -X POST -H "Content-Type: application/json" --data '[5.1,3.5,1.4,0.2]' http://localhost:8080/predict
```

In [55]:
# Setting environment variables in notebook
import os
os.environ['WANDB_API_KEY'] = 'fbd5089fe8ff8a2f6dfc053ad7ac625ab10a7a2f'
print('✓ WANDB_API_KEY set')

✓ WANDB_API_KEY set


In [57]:
# Set your artifact reference - update with your entity after training
import os
import wandb

# Use the correct entity where artifacts are stored
entity = 'ir2023'  # Change this to your entity if different

# Set the artifact reference
os.environ['WANDB_MODEL_ARTIFACT'] = f'{entity}/classroom-deploy/iris-rf:latest'
print(f'✓ WANDB_MODEL_ARTIFACT set to: {os.environ["WANDB_MODEL_ARTIFACT"]}')

✓ WANDB_MODEL_ARTIFACT set to: ir2023/classroom-deploy/iris-rf:latest


In [ ]:
import subprocess
import os
import time

# Make sure environment variables are set
os.environ['WANDB_API_KEY'] = 'fbd5089fe8ff8a2f6dfc053ad7ac625ab10a7a2f'

# Get username for artifact reference
import wandb
api = wandb.Api()
user = api.viewer
username = user.username if hasattr(user, 'username') else user.entity
os.environ['WANDB_MODEL_ARTIFACT'] = f'{username}/classroom-deploy/iris-rf:latest'

print(f'Starting FastAPI server with artifact: {os.environ["WANDB_MODEL_ARTIFACT"]}')
print('Server will start on http://localhost:8080')
print('\nNote: This will run in the background. To stop it, restart the kernel.')
print('Starting server...')

# Start uvicorn server in background (you'll need to run this in terminal for better control)

Starting FastAPI server with artifact: 112201022/classroom-deploy/iris-rf:latest
Server will start on http://localhost:8080

Note: This will run in the background. To stop it, restart the kernel.
Starting server...


: 

: 

In [ ]:
# Check if server is running
!curl -s http://localhost:8080/ || echo "\n❌ Server not running! Start it in a terminal first (see commands above)"

\n❌ Server not running! Start it in a terminal first (see commands above)


: 

: 

#### Check if the server is running:

Before testing with curl, verify the server is up:

In [58]:
import wandb
import os

# Get correct entity
entity = 'ir2023'  # Use the entity where artifacts are stored

# Get current working directory
cwd = os.getcwd()

commands = f"""
╔════════════════════════════════════════════════════════════════════╗
║           Commands to Start FastAPI Server in Terminal             ║
╚════════════════════════════════════════════════════════════════════╝

1. Open a NEW TERMINAL (Ctrl+Shift+`)

2. Copy and paste these commands:

cd {cwd}
export WANDB_API_KEY='fbd5089fe8ff8a2f6dfc053ad7ac625ab10a7a2f'
export WANDB_MODEL_ARTIFACT='{entity}/classroom-deploy/iris-rf:latest'
uvicorn app.main:app --host 0.0.0.0 --port 8080

3. You should see:
   INFO:     Started server process
   INFO:     Uvicorn running on http://0.0.0.0:8080

4. Keep that terminal running and come back to this notebook to test!

═══════════════════════════════════════════════════════════════════════
"""

print(commands)


╔════════════════════════════════════════════════════════════════════╗
║           Commands to Start FastAPI Server in Terminal             ║
╚════════════════════════════════════════════════════════════════════╝

1. Open a NEW TERMINAL (Ctrl+Shift+`)

2. Copy and paste these commands:

cd /home/chakri/Documents/S-7/MLoPs/Mlops2025w-112201022/CLASS/Week-8
export WANDB_API_KEY='fbd5089fe8ff8a2f6dfc053ad7ac625ab10a7a2f'
export WANDB_MODEL_ARTIFACT='112201022/classroom-deploy/iris-rf:latest'
uvicorn app.main:app --host 0.0.0.0 --port 8080

3. You should see:
   INFO:     Started server process
   INFO:     Uvicorn running on http://0.0.0.0:8080

4. Keep that terminal running and come back to this notebook to test!

═══════════════════════════════════════════════════════════════════════



#### Copy and run these commands in a separate terminal:

Run the cell below to get the exact commands you need to copy to your terminal.

### 4.3 Start the FastAPI Server

To test the FastAPI app, you need to start the server first. You have two options:

**Option 1: Run in Terminal (Recommended)**
Open a new terminal and run:
```bash
cd /home/chakri/Documents/S-7/MLoPs/Mlops2025w-112201022/CLASS/Week-8
export WANDB_API_KEY='fbd5089fe8ff8a2f6dfc053ad7ac625ab10a7a2f'
export WANDB_MODEL_ARTIFACT='your-username/classroom-deploy/iris-rf:latest'
uvicorn app.main:app --host 0.0.0.0 --port 8080
```

**Option 2: Run in Background from Notebook**
Use the cell below to start the server in the background.

In [ ]:
# Test the FastAPI server
# Make sure the server is running first (see previous cells)
!curl -X POST -H "Content-Type: application/json" --data '[5.1,3.5,1.4,0.2]' http://localhost:8080/predict

curl: (7) Failed to connect to localhost port 8080 after 0 ms: Couldn't connect to server


: 

: 

In [ ]:
# Alternative: Test with Python requests
import requests
import json

try:
    # Test root endpoint
    response = requests.get('http://localhost:8080/')
    print('✅ Server Status:', response.json())
    
    # Test prediction endpoint
    data = [5.1, 3.5, 1.4, 0.2]  # Iris sample
    response = requests.post('http://localhost:8080/predict', json=data)
    print('\n✅ Prediction Result:', response.json())
    print(f'\nPredicted Iris class: {response.json()["prediction"]}')
    
except requests.exceptions.ConnectionError:
    print('❌ Error: Cannot connect to server!')
    print('Make sure the FastAPI server is running in a terminal.')
    print('Run the terminal commands from the cells above.')

❌ Error: Cannot connect to server!
Make sure the FastAPI server is running in a terminal.
Run the terminal commands from the cells above.


: 

: 

### 4.4 Test the API (after server is running)

Once your server is running, test it with curl or Python requests:

## 5) Dockerize the app — build and run on local Ubuntu lab machines

### 5.1 Get Your Artifact Reference

**IMPORTANT:** First, get your correct W&B entity/username from the cell below.

In [59]:
import wandb
import os

# Get your username/entity
api = wandb.Api()
user = api.viewer
entity = user.entity  # This is the correct entity name for artifacts

print(f"Your W&B Entity: {entity}")
print(f"Your Artifact Reference: {entity}/classroom-deploy/iris-rf:latest")
print("\n" + "="*70)
print("DOCKER COMMANDS - Copy and run in terminal:")
print("="*70)

commands = f"""
# 1. Build the Docker image
cd /home/chakri/Documents/S-7/MLoPs/Mlops2025w-112201022/CLASS/Week-8/app
docker build -t iris-wandb:local .

# 2. Run the container
docker run -d \\
  --name iris-wandb-container \\
  -e WANDB_API_KEY='fbd5089fe8ff8a2f6dfc053ad7ac625ab10a7a2f' \\
  -e WANDB_MODEL_ARTIFACT='{entity}/classroom-deploy/iris-rf:latest' \\
  -p 8080:8080 \\
  iris-wandb:local

# 3. Wait for startup and check logs
sleep 10
docker logs iris-wandb-container

# 4. Test with curl
curl http://localhost:8080/
curl -X POST -H "Content-Type: application/json" --data '[5.1,3.5,1.4,0.2]' http://localhost:8080/predict

# 5. Stop and remove container (when done)
docker stop iris-wandb-container
docker rm iris-wandb-container
"""

print(commands)

Your W&B Entity: ir2023
Your Artifact Reference: ir2023/classroom-deploy/iris-rf:latest

DOCKER COMMANDS - Copy and run in terminal:

# 1. Build the Docker image
cd /home/chakri/Documents/S-7/MLoPs/Mlops2025w-112201022/CLASS/Week-8/app
docker build -t iris-wandb:local .

# 2. Run the container
docker run -d \
  --name iris-wandb-container \
  -e WANDB_API_KEY='fbd5089fe8ff8a2f6dfc053ad7ac625ab10a7a2f' \
  -e WANDB_MODEL_ARTIFACT='ir2023/classroom-deploy/iris-rf:latest' \
  -p 8080:8080 \
  iris-wandb:local

# 3. Wait for startup and check logs
sleep 10
docker logs iris-wandb-container

# 4. Test with curl
curl http://localhost:8080/
curl -X POST -H "Content-Type: application/json" --data '[5.1,3.5,1.4,0.2]' http://localhost:8080/predict

# 5. Stop and remove container (when done)
docker stop iris-wandb-container
docker rm iris-wandb-container



In [64]:
# Test the Dockerized API
import requests
import json

base_url = "http://localhost:8080"

print("═" * 70)
print("TESTING DOCKERIZED API")
print("═" * 70)

try:
    # Test root endpoint
    print("\n1. Testing root endpoint (GET /)...")
    response = requests.get(f"{base_url}/", timeout=5)
    print(f"   Status: {response.status_code}")
    print(f"   Response: {json.dumps(response.json(), indent=2)}")
    
    # Test predictions
    print("\n2. Testing predictions (POST /predict)...")
    
    test_samples = [
        {"data": [5.1, 3.5, 1.4, 0.2], "name": "Setosa"},
        {"data": [6.7, 3.0, 5.2, 2.3], "name": "Virginica"},
        {"data": [5.9, 3.0, 4.2, 1.5], "name": "Versicolor"},
    ]
    
    iris_names = ["Setosa", "Versicolor", "Virginica"]
    
    for sample in test_samples:
        response = requests.post(
            f"{base_url}/predict",
            json=sample['data'],
            timeout=5
        )
        result = response.json()
        prediction = result.get('prediction')
        print(f"\n   Input: {sample['data']}")
        print(f"   Prediction: {prediction} ({iris_names[prediction]})")
        print(f"   Expected: {sample['name']}")
        print(f"   ✅ Match!" if iris_names[prediction] == sample['name'] else "   ❌ Mismatch")
    
    print("\n" + "═" * 70)
    print("✅ ALL DOCKER TESTS PASSED!")
    print("═" * 70)
    
except requests.exceptions.ConnectionError:
    print("\n❌ Cannot connect to Docker container!")
    print("   Make sure the container is running: docker ps | grep iris-wandb")
except Exception as e:
    print(f"\n❌ Error: {e}")

══════════════════════════════════════════════════════════════════════
TESTING DOCKERIZED API
══════════════════════════════════════════════════════════════════════

1. Testing root endpoint (GET /)...
   Status: 200
   Response: {
  "status": "ok",
  "model_artifact": "ir2023/classroom-deploy/iris-rf:latest"
}

2. Testing predictions (POST /predict)...

   Input: [5.1, 3.5, 1.4, 0.2]
   Prediction: 0 (Setosa)
   Expected: Setosa
   ✅ Match!

   Input: [6.7, 3.0, 5.2, 2.3]
   Prediction: 2 (Virginica)
   Expected: Virginica
   ✅ Match!

   Input: [5.9, 3.0, 4.2, 1.5]
   Prediction: 1 (Versicolor)
   Expected: Versicolor
   ✅ Match!

══════════════════════════════════════════════════════════════════════
✅ ALL DOCKER TESTS PASSED!
══════════════════════════════════════════════════════════════════════


### 5.4 Test Dockerized API

Test the API running in the Docker container.

In [63]:
# Check if Docker container is running
import subprocess

result = subprocess.run(
    ["docker", "ps", "--filter", "name=iris-wandb"],
    capture_output=True,
    text=True
)

print("Docker Containers:")
print(result.stdout)

if "iris-wandb" in result.stdout and "Up" in result.stdout:
    print("\n✅ Container is running!")
    
    # Get container logs
    logs = subprocess.run(
        ["docker", "logs", "--tail", "20", "iris-wandb-container"],
        capture_output=True,
        text=True
    )
    print("\nRecent logs:")
    print(logs.stdout[-500:] if len(logs.stdout) > 500 else logs.stdout)
else:
    print("\n⚠️ Container not running. Run the Docker run command first.")

Docker Containers:
CONTAINER ID   IMAGE              COMMAND                  CREATED          STATUS          PORTS                                       NAMES
31f770bd25c9   iris-wandb:local   "uvicorn main:app --…"   54 minutes ago   Up 54 minutes   0.0.0.0:8080->8080/tcp, :::8080->8080/tcp   iris-wandb-test


✅ Container is running!

Recent logs:



In [62]:
# Generate Docker run command with your artifact
import wandb

api = wandb.Api()
user = api.viewer
entity = user.entity
artifact_ref = f'{entity}/classroom-deploy/iris-rf:latest'

docker_run_cmd = f"""
═══════════════════════════════════════════════════════════════════════
DOCKER RUN COMMAND - Copy and paste in terminal:
═══════════════════════════════════════════════════════════════════════

# Stop any existing container
docker stop iris-wandb-container 2>/dev/null || true
docker rm iris-wandb-container 2>/dev/null || true

# Run the container
docker run -d \\
  --name iris-wandb-container \\
  -e WANDB_API_KEY='fbd5089fe8ff8a2f6dfc053ad7ac625ab10a7a2f' \\
  -e WANDB_MODEL_ARTIFACT='{artifact_ref}' \\
  -p 8080:8080 \\
  iris-wandb:local

# Wait for startup
echo "Waiting for container to start..."
sleep 10

# Check logs
docker logs iris-wandb-container

# Check if running
docker ps | grep iris-wandb

═══════════════════════════════════════════════════════════════════════
"""

print(docker_run_cmd)


═══════════════════════════════════════════════════════════════════════
DOCKER RUN COMMAND - Copy and paste in terminal:
═══════════════════════════════════════════════════════════════════════

# Stop any existing container
docker stop iris-wandb-container 2>/dev/null || true
docker rm iris-wandb-container 2>/dev/null || true

# Run the container
docker run -d \
  --name iris-wandb-container \
  -e WANDB_API_KEY='fbd5089fe8ff8a2f6dfc053ad7ac625ab10a7a2f' \
  -e WANDB_MODEL_ARTIFACT='ir2023/classroom-deploy/iris-rf:latest' \
  -p 8080:8080 \
  iris-wandb:local

# Wait for startup
echo "Waiting for container to start..."
sleep 10

# Check logs
docker logs iris-wandb-container

# Check if running
docker ps | grep iris-wandb

═══════════════════════════════════════════════════════════════════════



### 5.3 Run Docker Container Locally

Run the Docker container on your Ubuntu machine.

In [61]:
# Verify Docker image was built
import subprocess

result = subprocess.run(
    ["docker", "images", "iris-wandb"],
    capture_output=True,
    text=True
)

print("Docker Images:")
print(result.stdout)

if "iris-wandb" in result.stdout:
    print("\n✅ Docker image 'iris-wandb:local' built successfully!")
else:
    print("\n❌ Image not found. Run the build command first.")

Docker Images:
REPOSITORY   TAG       IMAGE ID       CREATED          SIZE
iris-wandb   local     fd6a275c7e61   55 minutes ago   488MB


✅ Docker image 'iris-wandb:local' built successfully!


In [60]:
# Build Docker image - Run this in terminal instead
# This cell shows the command, but execute it in a terminal

import subprocess
import os

print("═" * 70)
print("DOCKER BUILD COMMAND")
print("═" * 70)
print("\nCopy and run this command in a terminal:\n")
print("cd /home/chakri/Documents/S-7/MLoPs/Mlops2025w-112201022/CLASS/Week-8/app")
print("docker build -t iris-wandb:local .")
print("\nOr run it from here:")

# Uncomment to build from notebook (may take a few minutes)
# result = subprocess.run(
#     ["docker", "build", "-t", "iris-wandb:local", "app/"],
#     capture_output=True,
#     text=True
# )
# print(result.stdout)
# print(result.stderr)

══════════════════════════════════════════════════════════════════════
DOCKER BUILD COMMAND
══════════════════════════════════════════════════════════════════════

Copy and run this command in a terminal:

cd /home/chakri/Documents/S-7/MLoPs/Mlops2025w-112201022/CLASS/Week-8/app
docker build -t iris-wandb:local .

Or run it from here:


### 5.2 Build Docker Image

Now build the Docker image from your FastAPI app.

## 6) Deploy to Google Cloud Run

Google Cloud Run is a fully managed platform that runs containers. It's perfect for deploying ML models because:
- **Serverless**: Pay only when requests are being processed
- **Auto-scaling**: Scales from 0 to N instances
- **Easy deployment**: Simple gcloud commands
- **HTTPS by default**: Automatic SSL certificates

### Prerequisites:

1. **Google Cloud Account** with billing enabled
2. **gcloud CLI** installed and configured
3. **Docker** working locally (we already have this!)
4. **Project created** in Google Cloud Console

### 6.1 Setup Google Cloud CLI

If you haven't installed gcloud CLI yet, follow these steps:

In [70]:
management_commands = """
═══════════════════════════════════════════════════════════════════════
CLOUD RUN MANAGEMENT COMMANDS
═══════════════════════════════════════════════════════════════════════

# List all Cloud Run services
gcloud run services list

# Get service details
gcloud run services describe iris-wandb --region us-central1

# View logs
gcloud run services logs read iris-wandb --region us-central1

# Follow logs in real-time
gcloud run services logs tail iris-wandb --region us-central1

# Update environment variables
gcloud run services update iris-wandb \\
  --region us-central1 \\
  --set-env-vars WANDB_MODEL_ARTIFACT=ir2023/classroom-deploy/iris-rf:v1

# Update to new image version
gcloud run deploy iris-wandb \\
  --image gcr.io/YOUR_PROJECT_ID/iris-wandb:v2 \\
  --region us-central1

# Delete the service (cleanup)
gcloud run services delete iris-wandb --region us-central1

# View metrics and monitoring
# Go to: https://console.cloud.google.com/run

═══════════════════════════════════════════════════════════════════════
COST OPTIMIZATION TIPS
═══════════════════════════════════════════════════════════════════════

1. Use min-instances 0 to scale to zero when not in use
2. Set appropriate CPU and memory (1 vCPU + 1Gi is usually enough)
3. Set timeout to minimum needed (300s for initial model download)
4. Monitor usage in Cloud Console
5. Delete service when not needed

Cloud Run pricing (as of 2025):
- Free tier: 2 million requests/month
- Pay only for actual request processing time
- Scales to zero = $0 when idle!

═══════════════════════════════════════════════════════════════════════
"""

print(management_commands)

with open('gcp_deploy_commands.txt', 'a') as f:
    f.write('\n' + management_commands)
    
print("\n✅ All GCP commands saved to gcp_deploy_commands.txt")


═══════════════════════════════════════════════════════════════════════
CLOUD RUN MANAGEMENT COMMANDS
═══════════════════════════════════════════════════════════════════════

# List all Cloud Run services
gcloud run services list

# Get service details
gcloud run services describe iris-wandb --region us-central1

# View logs
gcloud run services logs read iris-wandb --region us-central1

# Follow logs in real-time
gcloud run services logs tail iris-wandb --region us-central1

# Update environment variables
gcloud run services update iris-wandb \
  --region us-central1 \
  --set-env-vars WANDB_MODEL_ARTIFACT=ir2023/classroom-deploy/iris-rf:v1

# Update to new image version
gcloud run deploy iris-wandb \
  --image gcr.io/YOUR_PROJECT_ID/iris-wandb:v2 \
  --region us-central1

# Delete the service (cleanup)
gcloud run services delete iris-wandb --region us-central1

# View metrics and monitoring
# Go to: https://console.cloud.google.com/run

═══════════════════════════════════════════════

### 6.6 Cloud Run Management Commands

Useful commands for managing your Cloud Run service:

In [69]:
# Test Cloud Run deployment
# Replace CLOUD_RUN_URL with your actual Cloud Run URL after deployment

import requests
import json

# Replace this with your actual Cloud Run URL
CLOUD_RUN_URL = "https://iris-wandb-HASH-uc.a.run.app"  # Change this!

print("═" * 70)
print("TESTING CLOUD RUN DEPLOYMENT")
print("═" * 70)
print(f"\nURL: {CLOUD_RUN_URL}")
print("\n⚠️  UPDATE THE CLOUD_RUN_URL VARIABLE ABOVE WITH YOUR ACTUAL URL")
print("    You get this URL after running the deployment command")
print("\n" + "═" * 70)

# Uncomment and run after you have your Cloud Run URL:
"""
try:
    # Test root endpoint
    print("\n1. Testing root endpoint...")
    response = requests.get(f"{CLOUD_RUN_URL}/", timeout=30)
    print(f"   Status: {response.status_code}")
    print(f"   Response: {json.dumps(response.json(), indent=2)}")
    
    # Test prediction
    print("\n2. Testing prediction endpoint...")
    test_data = [5.1, 3.5, 1.4, 0.2]
    response = requests.post(
        f"{CLOUD_RUN_URL}/predict",
        json=test_data,
        timeout=30
    )
    result = response.json()
    iris_names = ["Setosa", "Versicolor", "Virginica"]
    prediction = result.get('prediction')
    
    print(f"   Input: {test_data}")
    print(f"   Prediction: {prediction} ({iris_names[prediction]})")
    print("\n✅ Cloud Run deployment is working!")
    
except Exception as e:
    print(f"\n❌ Error: {e}")
    print("   Make sure you've updated CLOUD_RUN_URL with your actual URL")
"""

══════════════════════════════════════════════════════════════════════
TESTING CLOUD RUN DEPLOYMENT
══════════════════════════════════════════════════════════════════════

URL: https://iris-wandb-HASH-uc.a.run.app

⚠️  UPDATE THE CLOUD_RUN_URL VARIABLE ABOVE WITH YOUR ACTUAL URL
    You get this URL after running the deployment command

══════════════════════════════════════════════════════════════════════


'\ntry:\n    # Test root endpoint\n    print("\n1. Testing root endpoint...")\n    response = requests.get(f"{CLOUD_RUN_URL}/", timeout=30)\n    print(f"   Status: {response.status_code}")\n    print(f"   Response: {json.dumps(response.json(), indent=2)}")\n\n    # Test prediction\n    print("\n2. Testing prediction endpoint...")\n    test_data = [5.1, 3.5, 1.4, 0.2]\n    response = requests.post(\n        f"{CLOUD_RUN_URL}/predict",\n        json=test_data,\n        timeout=30\n    )\n    result = response.json()\n    iris_names = ["Setosa", "Versicolor", "Virginica"]\n    prediction = result.get(\'prediction\')\n\n    print(f"   Input: {test_data}")\n    print(f"   Prediction: {prediction} ({iris_names[prediction]})")\n    print("\n✅ Cloud Run deployment is working!")\n\nexcept Exception as e:\n    print(f"\n❌ Error: {e}")\n    print("   Make sure you\'ve updated CLOUD_RUN_URL with your actual URL")\n'

### 6.5 Test Cloud Run Deployment

Once deployed, test your Cloud Run service:

In [68]:
# Generate Cloud Run deployment command
import subprocess
import wandb

# Get W&B artifact info
api = wandb.Api()
user = api.viewer
entity = user.entity
artifact_ref = f'{entity}/classroom-deploy/iris-rf:latest'

# Try to get GCP project ID
try:
    result = subprocess.run(
        ["gcloud", "config", "get-value", "project"],
        capture_output=True,
        text=True,
        timeout=5
    )
    project_id = result.stdout.strip()
    if not project_id or project_id == "(unset)":
        project_id = "YOUR_PROJECT_ID"
except:
    project_id = "YOUR_PROJECT_ID"

deploy_commands = f"""
═══════════════════════════════════════════════════════════════════════
DEPLOY TO GOOGLE CLOUD RUN
═══════════════════════════════════════════════════════════════════════

gcloud run deploy iris-wandb \\
  --image gcr.io/{project_id}/iris-wandb:latest \\
  --platform managed \\
  --region us-central1 \\
  --allow-unauthenticated \\
  --set-env-vars WANDB_API_KEY=fbd5089fe8ff8a2f6dfc053ad7ac625ab10a7a2f,WANDB_MODEL_ARTIFACT={artifact_ref} \\
  --memory 1Gi \\
  --cpu 1 \\
  --min-instances 0 \\
  --max-instances 10 \\
  --timeout 300

═══════════════════════════════════════════════════════════════════════

Options explained:
--image               : Your container image from GCR
--platform managed    : Use fully managed Cloud Run
--region             : us-central1 (or your preferred region)
--allow-unauthenticated : Make it publicly accessible
--set-env-vars       : Set environment variables (API key & artifact)
--memory 1Gi         : Allocate 1GB RAM
--cpu 1              : Allocate 1 vCPU
--min-instances 0    : Scale to zero when not in use (save costs!)
--max-instances 10   : Maximum concurrent instances
--timeout 300        : 5 minute timeout (for model download at startup)

═══════════════════════════════════════════════════════════════════════

After deployment, you'll get a URL like:
https://iris-wandb-HASH-uc.a.run.app

═══════════════════════════════════════════════════════════════════════
"""

print(deploy_commands)

# Append to commands file
with open('gcp_deploy_commands.txt', 'a') as f:
    f.write('\n' + deploy_commands)
    
print("\n✅ Commands appended to gcp_deploy_commands.txt")


═══════════════════════════════════════════════════════════════════════
DEPLOY TO GOOGLE CLOUD RUN
═══════════════════════════════════════════════════════════════════════

gcloud run deploy iris-wandb \
  --image gcr.io/YOUR_PROJECT_ID/iris-wandb:latest \
  --platform managed \
  --region us-central1 \
  --allow-unauthenticated \
  --set-env-vars WANDB_API_KEY=fbd5089fe8ff8a2f6dfc053ad7ac625ab10a7a2f,WANDB_MODEL_ARTIFACT=ir2023/classroom-deploy/iris-rf:latest \
  --memory 1Gi \
  --cpu 1 \
  --min-instances 0 \
  --max-instances 10 \
  --timeout 300

═══════════════════════════════════════════════════════════════════════

Options explained:
--image               : Your container image from GCR
--platform managed    : Use fully managed Cloud Run
--region             : us-central1 (or your preferred region)
--allow-unauthenticated : Make it publicly accessible
--set-env-vars       : Set environment variables (API key & artifact)
--memory 1Gi         : Allocate 1GB RAM
--cpu 1           

### 6.4 Deploy to Cloud Run

Once your image is in GCR, deploy it to Cloud Run:

In [67]:
# Generate GCR push commands with your project
import subprocess
import wandb

# Get W&B artifact info
api = wandb.Api()
user = api.viewer
entity = user.entity
artifact_ref = f'{entity}/classroom-deploy/iris-rf:latest'

# Try to get GCP project ID
try:
    result = subprocess.run(
        ["gcloud", "config", "get-value", "project"],
        capture_output=True,
        text=True,
        timeout=5
    )
    project_id = result.stdout.strip()
    if not project_id or project_id == "(unset)":
        project_id = "YOUR_PROJECT_ID"
except:
    project_id = "YOUR_PROJECT_ID"

gcr_commands = f"""
═══════════════════════════════════════════════════════════════════════
BUILD AND PUSH TO GOOGLE CONTAINER REGISTRY
═══════════════════════════════════════════════════════════════════════

PROJECT_ID: {project_id}
ARTIFACT: {artifact_ref}

─────────────────────────────────────────────────────────────────────
OPTION A: Build locally and push
─────────────────────────────────────────────────────────────────────

# Tag your local image for GCR
docker tag iris-wandb:local gcr.io/{project_id}/iris-wandb:latest

# Push to GCR
docker push gcr.io/{project_id}/iris-wandb:latest

─────────────────────────────────────────────────────────────────────
OPTION B: Use Cloud Build (recommended - builds in cloud)
─────────────────────────────────────────────────────────────────────

cd /home/chakri/Documents/S-7/MLoPs/Mlops2025w-112201022/CLASS/Week-8/app

# Build and push using Cloud Build
gcloud builds submit --tag gcr.io/{project_id}/iris-wandb

# This command:
# - Uploads your code to GCS
# - Builds the Docker image in the cloud
# - Pushes to GCR automatically

═══════════════════════════════════════════════════════════════════════
"""

print(gcr_commands)

# Save to file for easy reference
with open('gcp_deploy_commands.txt', 'w') as f:
    f.write(gcr_commands)
    
print("\n✅ Commands saved to gcp_deploy_commands.txt")


═══════════════════════════════════════════════════════════════════════
BUILD AND PUSH TO GOOGLE CONTAINER REGISTRY
═══════════════════════════════════════════════════════════════════════

PROJECT_ID: YOUR_PROJECT_ID
ARTIFACT: ir2023/classroom-deploy/iris-rf:latest

─────────────────────────────────────────────────────────────────────
OPTION A: Build locally and push
─────────────────────────────────────────────────────────────────────

# Tag your local image for GCR
docker tag iris-wandb:local gcr.io/YOUR_PROJECT_ID/iris-wandb:latest

# Push to GCR
docker push gcr.io/YOUR_PROJECT_ID/iris-wandb:latest

─────────────────────────────────────────────────────────────────────
OPTION B: Use Cloud Build (recommended - builds in cloud)
─────────────────────────────────────────────────────────────────────

cd /home/chakri/Documents/S-7/MLoPs/Mlops2025w-112201022/CLASS/Week-8/app

# Build and push using Cloud Build
gcloud builds submit --tag gcr.io/YOUR_PROJECT_ID/iris-wandb

# This command:
# 

### 6.3 Build and Push to Google Container Registry

Two options to build and push your image to Google Container Registry (GCR):

**Option A: Build locally and push** (faster if you already have the image)
**Option B: Use Google Cloud Build** (builds in the cloud)

In [66]:
# Google Cloud Project Setup Commands
# Copy and run these in your terminal

setup_commands = """
═══════════════════════════════════════════════════════════════════════
GOOGLE CLOUD SETUP COMMANDS
═══════════════════════════════════════════════════════════════════════

1. Login to Google Cloud:
   gcloud auth login

2. Create a new project (or use existing):
   gcloud projects create iris-ml-deploy --name="Iris ML Deployment"
   
   # Or list existing projects:
   gcloud projects list

3. Set your project:
   gcloud config set project YOUR_PROJECT_ID
   
   # Replace YOUR_PROJECT_ID with your actual project ID

4. Enable required APIs:
   gcloud services enable run.googleapis.com
   gcloud services enable containerregistry.gcr.io
   gcloud services enable cloudbuild.googleapis.com

5. Configure Docker authentication:
   gcloud auth configure-docker

6. Set default region (optional):
   gcloud config set run/region us-central1

═══════════════════════════════════════════════════════════════════════
"""

print(setup_commands)

# Try to get current project
import subprocess
try:
    result = subprocess.run(
        ["gcloud", "config", "get-value", "project"],
        capture_output=True,
        text=True,
        timeout=5
    )
    project_id = result.stdout.strip()
    if project_id and project_id != "(unset)":
        print(f"\n✅ Current project: {project_id}")
    else:
        print("\n⚠️ No project set. Run the commands above to configure.")
except:
    print("\n⚠️ Run gcloud commands in terminal to configure.")


═══════════════════════════════════════════════════════════════════════
GOOGLE CLOUD SETUP COMMANDS
═══════════════════════════════════════════════════════════════════════

1. Login to Google Cloud:
   gcloud auth login

2. Create a new project (or use existing):
   gcloud projects create iris-ml-deploy --name="Iris ML Deployment"

   # Or list existing projects:
   gcloud projects list

3. Set your project:
   gcloud config set project YOUR_PROJECT_ID

   # Replace YOUR_PROJECT_ID with your actual project ID

4. Enable required APIs:
   gcloud services enable run.googleapis.com
   gcloud services enable containerregistry.gcr.io
   gcloud services enable cloudbuild.googleapis.com

5. Configure Docker authentication:
   gcloud auth configure-docker

6. Set default region (optional):
   gcloud config set run/region us-central1

═══════════════════════════════════════════════════════════════════════


⚠️ Run gcloud commands in terminal to configure.


### 6.2 Configure Google Cloud Project

Run these commands in a terminal:

In [65]:
# Check if gcloud is installed
import subprocess

try:
    result = subprocess.run(
        ["gcloud", "--version"],
        capture_output=True,
        text=True,
        timeout=5
    )
    print("✅ Google Cloud SDK is installed!")
    print(result.stdout)
except FileNotFoundError:
    print("❌ gcloud CLI not found!")
    print("\nInstall it with:")
    print("curl https://sdk.cloud.google.com | bash")
    print("exec -l $SHELL")
    print("\nOr visit: https://cloud.google.com/sdk/docs/install")
except Exception as e:
    print(f"Error: {e}")

❌ gcloud CLI not found!

Install it with:
curl https://sdk.cloud.google.com | bash
exec -l $SHELL

Or visit: https://cloud.google.com/sdk/docs/install


## 7) Optional: GitHub Actions for CI/CD (auto-deploy on push)

Create `.github/workflows/ci-cd.yml` with the following content (example for Cloud Run deployment):

```yaml
name: CI/CD
on:
  push:
    branches: [ main ]
jobs:
  build-deploy:
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@v4
      - uses: google-github-actions/setup-gcloud@v1
        with:
          project_id: ${{ secrets.GCP_PROJECT }}
          service_account_key: ${{ secrets.GCP_SA_KEY }}
      - name: Build & push
        run: |
          gcloud builds submit --tag gcr.io/$GCP_PROJECT/iris-wandb
      - name: Deploy to Cloud Run
        run: |
          gcloud run deploy iris-wandb \
            --image gcr.io/$GCP_PROJECT/iris-wandb \
            --platform managed \
            --region us-central1 \
            --allow-unauthenticated \
            --set-env-vars=WANDB_API_KEY=${{ secrets.WANDB_API_KEY }},WANDB_MODEL_ARTIFACT=your-username/classroom-deploy/iris-rf:latest
```

Use GitHub Secrets for `GCP_SA_KEY` and `WANDB_API_KEY`.

## 8) Troubleshooting & common issues

- **W&B authentication errors**: Ensure `WANDB_API_KEY` is set or run `wandb login`.
- **Artifact not found**: Check artifact path format: `username/project/artifact-name:version` or `username/project/artifact-name:latest`.
- **Docker permissions**: If `docker build` fails due to permissions, ensure your user can run Docker or use `sudo`.
- **Startup latency**: Downloading the artifact at container startup adds latency — consider baking the model into the image for production.



## Instructor notes & grading rubric

Suggested rubric:
- Model logged as W&B Artifact: 10 pts
- Local FastAPI app runs and predicts (no Docker): 20 pts
- Docker image build successful: 20 pts
- Cloud Run (or cloud) deployment functional: 30 pts
- README / Documentation clarity: 20 pts

Instructor tip: Pre-create a W&B project and share artifact refs to reduce setup noise in class.
